# ASX Companies Database

In [35]:
# dependencies
from splinter import Browser
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
from sqlalchemy import create_engine
import time

## Table 1: ASX 200 Company

## Extract data from web scraping, source: https://www.marketindex.com.au/asx200

In [36]:
# setup splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

In [37]:
# define url
url = 'https://www.marketindex.com.au/asx200'

In [38]:
# visit url
browser.visit(url)

In [39]:
# wait for 5 seconds
time.sleep(5)

In [40]:
# click 'Show All Companies' button
target = 'button[class="btn control-company-display"]'
browser.find_by_tag(target).click()

In [41]:
# wait for 5 seconds
time.sleep(5)

In [42]:
# HTML object
html = browser.html
# Parse HTML with Beautiful Soup
soup = BeautifulSoup(html, 'html.parser')

In [43]:
# check classes of tables
print('Classes of tables:')
for table in soup.find_all('table'):
    print(table.get('class'))

Classes of tables:
['mi-table']
['mt-3', 'mi-table']
['mi-table']
['mi-table', 'mb-4', 'quoteapi-even-items']
['mi-table', 'mb-4', 'quoteapi-odd-items']


In [44]:
#  retrieve data from table with class 'mi-table', 'mb-4', 'quoteapi-even-items'
table = soup.find('table', class_='mi-table mb-4 quoteapi-even-items')

In [45]:
# define dataframe
asx200_df = pd.DataFrame(columns=['company_code', 'company_name', 'price_29jul22', 'change', 'percent_change', 
                                  'high', 'low', 'volume', 'market_cap', 'one_year_percent_change'])

# collect data
for row in table.tbody.find_all('tr'):    
    # Find all data for each column
    columns = row.find_all('td')
    
    if(columns != []):
        company_code = columns[1].text
        company_name = columns[2].span.contents[0]
        price_29jul22 = columns[3].text
        change = columns[4].text
        percent_change = columns[5].text
        high = columns[6].text
        low = columns[7].text
        volume = columns[8].text
        market_cap = columns[9].text
        one_year_percent_change = columns[10].text

        asx200_df = asx200_df.append({'company_code': company_code,  'company_name': company_name, 'price_29jul22': price_29jul22, 
                                      'change': change, 'percent_change': percent_change, 'high': high, 'low':low, 'volume':volume,
                                      'market_cap': market_cap, 'one_year_percent_change': one_year_percent_change }, ignore_index=True)

In [64]:
# show data
print(len(asx200_df))
asx200_df.head()

200


,company_code,company_name,price_29jul22,change,percent_change,high,low,volume,market_cap,one_year_percent_change
0,A200,Betashares Australia 200 ETF,$115.50,+1.03,+0.90%,$115.91,$115.39,"32,136",$2.2B,-7.75%
1,A2M,The a2 Milk Company Ltd,$4.54,+0.14,+3.18%,$4.54,$4.38,"2,779,566",$3.4B,-26.06%
2,AAA,Betashares Australian High Interest Cash ETF,$50.13,0.00,0.00,$50.13,$50.12,"656,478",$1.8B,+0.08%
3,ABP,Abacus Property Group,$2.93,+0.05,+1.74%,$2.95,$2.91,"986,196",$2.6B,-6.39%
4,AFI,Australian Foundation Investment Company Ltd,$8.00,-0.02,-0.25%,$8.04,$7.99,"348,190",$9.8B,-4.08%


In [47]:
# end splinter
browser.quit()

## Transform data

In [48]:
# remove column
transformed_asx200_df = asx200_df[['company_code', 'company_name', 'price_29jul22', 'change','percent_change', 
                                   'one_year_percent_change']].copy()

# drop duplicates
transformed_asx200_df = transformed_asx200_df.drop_duplicates(subset=['company_code'])

# fill na
transformed_asx200_df= transformed_asx200_df.fillna(0)

print(len(transformed_asx200_df))

# check data type
transformed_asx200_df.dtypes

200


company_code               object
company_name               object
price_29jul22              object
change                     object
percent_change             object
one_year_percent_change    object
dtype: object

In [49]:
# remove $ signs
transformed_asx200_df['price_29jul22'] = transformed_asx200_df['price_29jul22'].map(lambda x: x.lstrip('$'))

In [50]:
# change %str to float
transformed_asx200_df['percent_change'] = transformed_asx200_df['percent_change'].str.rstrip('%').astype('float')
transformed_asx200_df['one_year_percent_change'] = transformed_asx200_df['one_year_percent_change'].str.rstrip('%').astype('float')

In [51]:
# change data type
transformed_asx200_df = transformed_asx200_df.astype({'price_29jul22':'float','change':'float'})

transformed_asx200_df.dtypes

company_code                object
company_name                object
price_29jul22              float64
change                     float64
percent_change             float64
one_year_percent_change    float64
dtype: object

In [52]:
# sort by company_code
transformed_asx200_df= transformed_asx200_df.sort_values(by=['company_code'])

In [65]:
# show data
print(len(transformed_asx200_df))
transformed_asx200_df.head()

200


,company_code,company_name,price_29jul22,change,percent_change,one_year_percent_change
0,A200,Betashares Australia 200 ETF,115.50,1.03,0.90,-7.75
1,A2M,The a2 Milk Company Ltd,4.54,0.14,3.18,-26.06
2,AAA,Betashares Australian High Interest Cash ETF,50.13,0.00,0.00,0.08
3,ABP,Abacus Property Group,2.93,0.05,1.74,-6.39
4,AFI,Australian Foundation Investment Company Ltd,8.00,-0.02,-0.25,-4.08


## Table 2: ASX Company

## Extract data from CSV file, source: https://www.listcorp.com/asx/

In [54]:
# create dataframe from CSV file
asx_company = "companies-list.csv"
asx_company_df = pd.read_csv(asx_company)
len(asx_company_df)

2217

In [66]:
# show data
print(len(asx_company_df))
asx_company_df.head()

2217


,Code,Company,Link,Market Cap,Last trade,Change,%Change,Sector
0,ASX:BHP,BHP Group Limited (ASX:BHP),https://www.listcorp.com/asx/bhp/bhp-group-lim...,193988000000,38.68,0.0,0.0,Materials
1,ASX:CBA,Commonwealth Bank (ASX:CBA),https://www.listcorp.com/asx/cba/commonwealth-...,169967000000,100.77,0.0,0.0,Financials
2,ASX:CSL,CSL Limited (ASX:CSL),https://www.listcorp.com/asx/csl/csl-limited,140369000000,289.84,0.0,0.0,Health Care
3,ASX:NAB,National Australia Bank (ASX:NAB),https://www.listcorp.com/asx/nab/nab,96384200000,30.60,0.0,0.0,Financials
4,ASX:WBC,Westpac Banking Corp (ASX:WBC),https://www.listcorp.com/asx/wbc/westpac,75029200000,21.51,0.0,0.0,Financials


## Transform data

In [56]:
# remove column
transformed_asx_company_df = asx_company_df[['Code', 'Company', 'Market Cap', 'Sector']].copy()

# change column name
transformed_asx_company_df.columns = ['company_code', 'company_name', 'market_cap', 'sector']

# drop duplicates
transformed_asx_company_df = transformed_asx_company_df.drop_duplicates(subset=['company_code'])

# drop na
transformed_asx_company_df = transformed_asx_company_df.dropna(thresh=2)

print(len(transformed_asx_company_df))

# check data type
transformed_asx_company_df.dtypes

2217


company_code    object
company_name    object
market_cap       int64
sector          object
dtype: object

In [57]:
# remame rows in column "company_code"
transformed_asx_company_df['company_code'] = transformed_asx_company_df['company_code'].map(lambda x: x.lstrip('ASX'))
transformed_asx_company_df.head()


,company_code,company_name,market_cap,sector
0,:BHP,BHP Group Limited (ASX:BHP),193988000000,Materials
1,:CBA,Commonwealth Bank (ASX:CBA),169967000000,Financials
2,:CSL,CSL Limited (ASX:CSL),140369000000,Health Care
3,:NAB,National Australia Bank (ASX:NAB),96384200000,Financials
4,:WBC,Westpac Banking Corp (ASX:WBC),75029200000,Financials


In [58]:
# remame rows in column "company_code"
transformed_asx_company_df['company_code'] = transformed_asx_company_df['company_code'].map(lambda x: x.lstrip(':'))

# sort by company_code
transformed_asx_company_df = transformed_asx_company_df.sort_values(by=['company_code'])

In [59]:
transformed_asx_company_df.head()

,company_code,company_name,market_cap,sector
1513,14D,1414 Degrees Limited (ASX:14D),21208500,Industrials
1636,1AD,AdAlta Limited (ASX:1AD),15709200,Health Care
1413,1AE,Aurora Energy Metals Limited (ASX:1AE),26270400,Materials
1777,1AG,Alterra Limited (ASX:1AG),11128800,Consumer Staples
902,1MC,Morella Corporation Limited (ASX:1MC),93171900,Materials


## Load dataframe to database 

In [60]:
# create connection
connection = "postgres:David$1986@localhost:5432/asx_db"
engine = create_engine(f'postgresql://{connection}')

In [61]:
# Confirm tables
engine.table_names()

c:\Users\david\Anaconda3\envs\bootcamp\lib\site-packages\ipykernel_launcher.py:2: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  


['asx_200', 'asx_companies']

In [62]:
# # load dataframe
transformed_asx200_df.to_sql(name="asx_200", con=engine, if_exists='append', index=False)

transformed_asx_company_df.to_sql(name="asx_companies", con=engine, if_exists='append', index=False)